In [54]:
import numpy as np 
from datetime import datetime, timedelta

countries = np.array(['Indonesia', 'Bolivia', 'Morocco', 'Bangladesh', 'Sri Lanka', 'Kenya', 'Mozambique',
                      "Afghanistan", "Côte d'Ivoire", "Pakistan", "El Salvador", 
                      "Philippines", "Uganda", "Kyrgyzstan", "Egypt", "Rwanda", "Zambia", "Angola", "Ghana", "Honduras"])

def import_posterior(country):
    post = np.load("./posteriors_october/posteriors_{0}.npz".format(country), allow_pickle=True)["arr_0"]
    posterior = {"R0": post[:, 0], "Delta": post[:, 1], "seasonality_min": post[:, 2], "psi": post[:, 3], 
                 "I_mult": post[:, 4], "R_mult": post[:, 5], "ifr_mult": post[:, 6], "scaler": 100*post[:, 7], "date_intro_VOC": post[:, 8]}
    return posterior

def get_stats(country, k):
    posterior = import_posterior(country)

    if k != "date_intro_VOC":
        return np.median(posterior[k]), np.quantile(posterior[k], q=0.25), np.quantile(posterior[k], q= 0.75)

    else: 
        ref_date = datetime(2020, 1, 1)
        delta_days = [(d - ref_date).days for d in posterior[k]]
        med, q1, q2 = np.median(delta_days), np.quantile(delta_days, q=0.25), np.quantile(delta_days, q= 0.75)
        
        return ref_date + timedelta(days=med), ref_date + timedelta(days=q1), ref_date + timedelta(days=q2)


In [87]:
countries =[]
            
params = ['R0', 'Delta', 'seasonality_min', 'psi', 'I_mult', 'R_mult', 'ifr_mult', 'scaler', 'date_intro_VOC']
labels = ["$R^{start}_{t}$", "$\Delta$",  "$\\alpha_{min}$", "$\psi$", "$Inf^{mult}_{start}$", "$Rec^{mult}_{start}$",
          "IFR Multiplier", "$\%$ deaths reported", "Date intro. VOC"]

countries_subs = ['Zambia', "Côte d'Ivoire", 'Kenya', 'Uganda']
for p, l in zip(params, labels):
    line = "\\textit{{{0}}}".format(l)

    for country in countries_subs:
        posterior = import_posterior(country)
        med, q1, q2 = get_stats(country, p)

        # format 
        if p == "Delta":
            med, q1, q2 = int(med), int(q1), int(q2)

        elif p != "date_intro_VOC":
            med, q1, q2 = np.round(med, 2), np.round(q1, 2), np.round(q2, 2)

        else:
            med, q1, q2 = med.strftime("%m-%d"), q1.strftime("%m-%d"), q2.strftime("%m-%d")

        line += " & {0} {{[}}{1}, {2}{{]}}".format(med, q1, q2)

    print(line + "\\\ ")

\textit{$R^{start}_{t}$} & 1.25 {[}1.24, 1.27{]} & 1.08 {[}1.06, 1.09{]} & 1.38 {[}1.37, 1.41{]} & 1.41 {[}1.39, 1.44{]}\\ 
\textit{$\Delta$} & 12 {[}11, 14{]} & 27 {[}19, 31{]} & 30 {[}27, 32{]} & 30 {[}28, 31{]}\\ 
\textit{$\alpha_{min}$} & 0.73 {[}0.62, 0.86{]} & 0.77 {[}0.64, 0.88{]} & 0.74 {[}0.62, 0.89{]} & 0.76 {[}0.64, 0.86{]}\\ 
\textit{$\psi$} & 1.13 {[}1.08, 1.2{]} & 1.09 {[}1.05, 1.15{]} & 1.41 {[}1.36, 1.47{]} & 1.11 {[}1.06, 1.18{]}\\ 
\textit{$Inf^{mult}_{start}$} & 16.27 {[}10.17, 21.0{]} & 16.07 {[}9.63, 20.61{]} & 20.5 {[}17.8, 22.96{]} & 1.62 {[}1.27, 2.14{]}\\ 
\textit{$Rec^{mult}_{start}$} & 13.13 {[}7.4, 19.23{]} & 13.19 {[}7.32, 19.05{]} & 13.02 {[}7.93, 19.22{]} & 13.84 {[}7.91, 19.86{]}\\ 
\textit{IFR Multiplier} & 1.05 {[}0.71, 1.46{]} & 1.0 {[}0.73, 1.39{]} & 0.99 {[}0.72, 1.38{]} & 1.04 {[}0.77, 1.43{]}\\ 
\textit{$\%$ deaths reported} & 21.87 {[}15.94, 31.72{]} & 2.11 {[}1.57, 2.96{]} & 6.58 {[}4.78, 9.31{]} & 42.36 {[}26.11, 63.94{]}\\ 
\textit{Date intro.